In [84]:
import random
import numpy as np
import gymnasium as gym

In [85]:
def Q(state, action, q_weights):
    return np.dot(np.append(state, action), q_weights)

In [86]:
def epsilon_greedy(env, obs, q_weights, epsilon):
    action = None
    e = random.random()
    if e < epsilon:
        action = env.action_space.sample()
    else:
        possible_actions = range(env.action_space.n)
        q_values = {Q(obs, a, q_weights) : a for a in possible_actions}
        action = q_values[max(q_values)]
    return action

In [87]:
def calculate_target(env, transition, q_weights, discount_rate):
    _, _, t_reward, t_new_obs, t_terminal = transition
    if t_terminal:
        target = t_reward
    else:
        possible_actions = range(env.action_space.n)
        max_Q = max([Q(t_new_obs, a, q_weights) for a in possible_actions])
        target = transition[2] + discount_rate * max_Q
    return target

In [88]:
# TODO
def gradient_descent(target, q_weights, obs, action, learning_rate):
    return q_weights

In [89]:
MAX_EPISODES = 5
MAX_TIMESTEPS = 50
MINIBATCH_SIZE = 5
EPSILON = .1
DISCOUNT_RATE = .9
LEARNING_RATE = .1

In [90]:
rng = np.random.default_rng()
env = gym.make("CartPole-v1", render_mode="human")
obs, info = env.reset()

In [91]:
replay_memory = []
q_weights = rng.random(env.observation_space.shape[0] + 1)

for episode in range(MAX_EPISODES):
    states = [obs]
    for t in range(MAX_TIMESTEPS):
        # sample and take action
        action = epsilon_greedy(env, obs, q_weights, EPSILON)
        new_obs, reward, terminated, truncated, info = env.step(action)
        # sample minibatch and update q_weights
        replay_memory.append([obs, action, reward, new_obs, terminated or truncated])
        minibatch = random.sample(replay_memory, min(MINIBATCH_SIZE, len(replay_memory)))
        for transition in minibatch:
            target = calculate_target(env, transition, q_weights, DISCOUNT_RATE)
            t_obs, t_action, _, _, _ = transition
            q_weights = gradient_descent(target, q_weights, t_obs, t_action, LEARNING_RATE)
        obs = new_obs

/Users/ben/.pyenv/versions/3.8.18/envs/drl/lib/python3.8/site-packages/gymnasium/envs/classic_control/cartpole.py:180: UserWarning: WARN: You are calling 'step()' even though this environment has already returned terminated = True. You should always call 'reset()' once you receive 'terminated = True' -- any further steps are undefined behavior.
  logger.warn(


In [92]:
env = gym.make("CartPole-v1", render_mode="human")
obs, _ = env.reset()
for _ in range(100):
    action = epsilon_greedy(env, obs, q_weights, EPSILON)
    obs, reward, terminated, truncated, info = env.step(action)
    if terminated or truncated:
        obs, _ = env.reset()

env.close()